# SABER Temperature Trend Analysis (1D, Silver Layer)

This notebook processes **bronze-level monthly SABER temperature data** 
and applies multiple statistical methods to detect trends in the Mesosphere-Lower Thermosphere (MLT).

**Pipeline stages:**
1. Setup imports and paths
2. Load helper functions
3. Load bronze-level input data
4. Run trend analyses:
   - Sequential Mann-Kendall test
   - Mann-Kendall test
   - Sen’s slope estimator
   - Innovative Trend Analysis (ITA)
5. Save outputs to the **silver layer


---
**Input:**  
- `bronze_saber_1d_tempoaral_agg.csv`

**Output:**  
- `silver_smk_1d.csv`  
- `silver_mk_1d.csv`  
- `silver_sen_slope_1d.csv`  
- `silver_ita_1d.csv`  
---

In [1]:
import pandas as pd
import numpy as np
import pymannkendall as mk
from scipy.stats import theilslopes, norm, linregress
import datetime as dt
import joblib
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Current working directory (notebook folder)
notebook_path = Path.cwd()  # MLT/code/data_processing
base_path = notebook_path.parent.parent  # Go up to MLT/

functions_path = notebook_path.parent / "000_Functions.ipynb"
data_path = base_path / "data" / "bronze"
silver_path = base_path / "data" / "silver"

In [3]:
# Load custom functions
%run "{functions_path.as_posix()}"

In [4]:
 # Read and concatenate all CSVs
df = pd.read_csv(data_path / "bronze_saber_1d_tempoaral_agg.csv.gz")

In [5]:
df['year_month'] = pd.to_datetime(df['year_month'], errors='coerce')
df['year_month'] = df['year_month'].dt.to_period('M')

## Run Trend Analyses

### Sequential Mann-Kendall Test
**Output:** silver_smk_1d.csv

In [6]:
smk_1d = run_sequential_mk(df)

### Mann-Kendall Test
**Output:** silver_mk_1d.csv

In [ ]:
mk_1d = run_mk_tests(df, mk)

### Sen’s Slope Estimation
**Output:** silver_sen_slope_1d.csv

In [ ]:
sen_slope = sen_slope(df)

### Innovative Trend Analysis (ITA)
**Output:** silver_ita_1d.csv

In [ ]:
ita = ITA(df)

## 6. Save Silver-layer Outputs

In [7]:
# Ensure silver folder exists
silver_path.mkdir(parents=True, exist_ok=True)

# Define tables to save
tables = {'silver_smk_1d':smk_1d,
          'silver_mk_1d':mk_1d, 
          'silver_sen_slope_1d':sen_slope, 
          'silver_ita_1d':ita}

# Save tables to CSV
for table_name, table in tables.items():
    try:
        file_path = silver_path / f"{table_name}.csv"
        table.to_csv(file_path, index=False)
        print(f"[SUCCESS] {table_name}.csv saved")
    except Exception as e:
        print(f"[ERROR] Could not save {table_name}. Reason: {e}")

print("All CSVs saved successfully.")

silver_smk_1d.csv saved.
silver_mk_1d.csv saved.
silver_sen_slope_1d.csv saved.
silver_ita_1d.csv saved.
All CSVs saved successfully.
